In [ ]:
DATASET = "/home/beisner/datasets/rlbench/"

In [ ]:
# Use RLBench to get the demo.
import numpy as np
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import JointVelocity
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.tasks import StackWine


action_mode = MoveArmThenGripper(
  arm_action_mode=JointVelocity(),
  gripper_action_mode=Discrete()
)
env = Environment(action_mode, DATASET, headless=True)
env.launch()

task = env.get_task(StackWine)

demos = task.get_demos(1)
env.shutdown()

In [ ]:
demos = np.array(demos)

In [ ]:
demos[0][0].overhead_point_cloud.shape

In [ ]:
import cv2
import io
from IPython.core import display
import tempfile


In [ ]:
import cv2
import io
import tempfile
import base64
from IPython.display import display, HTML, Image

# Write a function which takes a list of numpy RGB images and returns a video displayable in Jupyter.
def display_video(frames):
    height, width, layers = frames[0].shape

    # Create a temporary directory to store the video.
    with tempfile.TemporaryDirectory() as tmp:
        tmp = tmp + "/video.avi"
        video = cv2.VideoWriter(tmp, cv2.VideoWriter_fourcc(*"MJPG"), 30, (width, height))
        for frame in frames:
            video.write(frame)
        video.release()

        # Read the video into a bytestream which can be displayed in Jupyter.
        with open(tmp, "rb") as f:
            video_data = f.read()

    # Encode the video data in base64
    video_base64 = base64.b64encode(video_data).decode()
    
    # Generate HTML video tag
    video_tag = f"<video width='{width}' height='{height}' controls><source src='data:video/x-msvideo;base64,{video_base64}' type='video/avi'></video>"
    
    return display(HTML(video_tag))


In [ ]:
# Write a file which displays a single frame from a numpy RGB image without saving the image to disk.
def display_frame(frame):
    _, encoded_image = cv2.imencode('.png', frame)
    return display(Image(data=encoded_image))


In [ ]:
frames = [demo.overhead_rgb for demo in demos[0]]

In [ ]:
display_video(frames)

In [ ]:
display_frame(demos[0][0].overhead_rgb)

In [ ]:
def obs_to_rgb_point_cloud(obs):
    # Get the overhead, left, front, and right RGB images.
    overhead_rgb = obs.overhead_rgb
    left_rgb = obs.left_shoulder_rgb
    right_rgb = obs.right_shoulder_rgb
    front_rgb = obs.front_rgb

    # Get the overhead, left, front, and right point clouds. The point clouds are 
    # in the same shape as the images.
    overhead_point_cloud = obs.overhead_point_cloud
    left_point_cloud = obs.left_shoulder_point_cloud
    right_point_cloud = obs.right_shoulder_point_cloud
    front_point_cloud = obs.front_point_cloud

    # Get masks.
    overhead_mask = obs.overhead_mask
    left_mask = obs.left_shoulder_mask
    right_mask = obs.right_shoulder_mask
    front_mask = obs.front_mask

    # Flatten RGB and point cloud images into Nx3 arrays
    overhead_rgb = overhead_rgb.reshape((-1, 3))
    left_rgb = left_rgb.reshape((-1, 3))
    right_rgb = right_rgb.reshape((-1, 3))
    front_rgb = front_rgb.reshape((-1, 3))

    overhead_point_cloud = overhead_point_cloud.reshape((-1, 3))
    left_point_cloud = left_point_cloud.reshape((-1, 3))
    right_point_cloud = right_point_cloud.reshape((-1, 3))
    front_point_cloud = front_point_cloud.reshape((-1, 3))

    # Reshape the masks into Nx1 arrays.
    overhead_mask = overhead_mask.reshape((-1, 1))
    left_mask = left_mask.reshape((-1, 1))
    right_mask = right_mask.reshape((-1, 1))
    front_mask = front_mask.reshape((-1, 1))

    # Stack the RGB and point cloud images together.
    rgb = np.vstack((overhead_rgb, left_rgb, right_rgb, front_rgb))
    point_cloud = np.vstack((overhead_point_cloud, left_point_cloud, right_point_cloud, front_point_cloud))
    mask = np.vstack((overhead_mask, left_mask, right_mask, front_mask))

    return rgb, point_cloud, mask
    


In [ ]:
rgbs, point_clouds, masks = obs_to_rgb_point_cloud(demos[0][0])

In [ ]:
action_obj_ids = [160]
anchor_obj_ids = [152, 154]

# Get get rgb and point cloud for all points whose mask matches in list of ids
def get_rgb_point_cloud_by_mask(rgb, point_cloud, mask, ids):
    # Get the indices of the points which match the ids.
    indices = np.isin(mask, ids).reshape((-1))
    # Get the rgb and point cloud for the indices.
    rgb = rgb[indices]
    point_cloud = point_cloud[indices]
    return rgb, point_cloud

# Get the rgb and point cloud for the action objects.
action_rgb, action_point_cloud = get_rgb_point_cloud_by_mask(rgbs, point_clouds, masks, action_obj_ids)

# Get the rgb and point cloud for the anchor objects.
anchor_rgb, anchor_point_cloud = get_rgb_point_cloud_by_mask(rgbs, point_clouds, masks, anchor_obj_ids)

# Create an open3d visualization of these two point clouds
action_pc = o3d.geometry.PointCloud()
action_pc.points = o3d.utility.Vector3dVector(action_point_cloud)
action_pc.colors = o3d.utility.Vector3dVector(action_rgb / 255.0)

anchor_pc = o3d.geometry.PointCloud()
anchor_pc.points = o3d.utility.Vector3dVector(anchor_point_cloud)
anchor_pc.colors = o3d.utility.Vector3dVector(anchor_rgb / 255.0)

o3d.visualization.draw_geometries([action_pc, anchor_pc])

In [ ]:
import numpy as np

In [ ]:
import open3d as o3d

# Create an Open3D point cloud from the point cloud data, with RGB.
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(point_clouds)
# point_cloud.colors = o3d.utility.Vector3dVector(rgbs / 255.0)

# Masks contains per-class labels for each point. We can use this to color the point cloud.
# We don't know in advance how many classes there are, so we'll generate a random color for each class.
colors = np.random.uniform(0, 1, size=(masks.max() + 1, 3))
point_cloud.colors = o3d.utility.Vector3dVector(colors[masks[... , 0]])

# Visualize the point cloud.
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
# For each class in the mask, create a point cloud and a random color:
colors = np.random.uniform(0, 1, size=(len(np.unique(masks)), 3))
o3d_pcds = []
for cl, co in zip(np.unique(masks), colors):
    # Create a point cloud for this class.
    class_point_cloud = o3d.geometry.PointCloud()
    pts = point_clouds[masks[... , 0] == cl]
    class_point_cloud.points = o3d.utility.Vector3dVector(pts)
    # Repeat the color for each point in the point cloud.
    class_point_cloud.colors = o3d.utility.Vector3dVector(np.tile(co, (pts.shape[0], 1)))
    o3d_pcds.append(class_point_cloud)


In [ ]:
# Visualize the point clouds, one at a time
for masknum, pcd in zip(np.unique(masks), o3d_pcds):
    print(masknum)
    o3d.visualization.draw_geometries([pcd])


In [ ]:
# Create a plotly figure with the point clouds.
import plotly.graph_objects as go

fig = go.Figure()

# The labels are stored in the masks. We can use this to color the point cloud.

# fig.add_trace(go.Scatter3d(
#     x=point_clouds[... , 0],
#     y=point_clouds[... , 1],
#     z=point_clouds[... , 2],
#     mode='markers',
#     marker=dict(
#         size=2,
#         color=masks[... , 0],                # set color to an array/list of desired values
#         colorscale='Viridis',   # choose a colorscale
#         opacity=0.8
#     )
# ))

# For each class in the mask, create a point cloud and a random color:
colors = np.random.uniform(0, 1, size=(len(np.unique(masks)), 3))
for cl, co in zip(np.unique(masks), colors):
    # Create a point cloud for this class.
    pts = point_clouds[masks[... , 0] == cl]
    fig.add_trace(go.Scatter3d(
        x=pts[... , 0],
        y=pts[... , 1],
        z=pts[... , 2],
        mode='markers',
        marker=dict(
            size=2,
            color=co,                # set color to an array/list of desired values
            opacity=0.8
        ),
        name=str(cl)
    ))


fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1,1],),
        yaxis=dict(nticks=4, range=[-1,1],),
        zaxis=dict(nticks=4, range=[-1,3],),
    ),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

# Add a legend with the class names.
fig.update_layout(
    legend_title_text='Classes',
    legend_traceorder="reversed",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)


fig.show()

In [ ]:
np.unique(masks)

In [ ]:
colors[masks].shape

In [ ]:
np.unique(masks)

In [ ]:
dir(demos[0][0])

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from rpad.rlbench_utils.placement_dataset import RLBenchPlacementDataset

In [ ]:
dset = RLBenchPlacementDataset(DATASET, "stack_wine", 10)

In [ ]:
data = dset[0]

action_pc = data['action_pc']
action_rgb = data['action_rgb']

# Use open3d to visualize the point cloud.
o3d_action_pc = o3d.geometry.PointCloud()
o3d_action_pc.points = o3d.utility.Vector3dVector(action_pc)
o3d_action_pc.colors = o3d.utility.Vector3dVector(action_rgb / 255.0)
# o3d.visualization.draw_geometries([o3d_action_pc])


anchor_pc = data['anchor_pc']
anchor_rgb = data['anchor_rgb']

# Use open3d to visualize the point cloud.
o3d_anchor_pc = o3d.geometry.PointCloud()
o3d_anchor_pc.points = o3d.utility.Vector3dVector(anchor_pc)
o3d_anchor_pc.colors = o3d.utility.Vector3dVector(anchor_rgb / 255.0)
# o3d.visualization.draw_geometries([o3d_anchor_pc])

o3d.visualization.draw_geometries([o3d_action_pc, o3d_anchor_pc])


In [ ]:
# Draw both


In [ ]:
action_rgb.shape